# Implementation of TextRank
(Based on: https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf)

The input text is given below

In [1]:
file_path = 'doc_1.txt'
with open(file_path, 'r') as f:
    text = f.readlines()
Text = [c.strip('\n') for c in text if len(c)>40]
Text = ' '.join(Text)

In [37]:
from gensim.summarization import keywords
from gensim.summarization.keywords import get_graph
from gensim.summarization.pagerank_weighted import pagerank_weighted


In [41]:
graph = get_graph(Text)

In [46]:
import operator
result = pagerank_weighted(graph)
sorted_x = sorted(result.items(),reverse=True,key=operator.itemgetter(1))

In [48]:
import pandas as pd

In [49]:
df = pd.DataFrame(sorted_x)

In [53]:
df[1].sum()

15.062458327288375

In [36]:
keywords(Text,ratio=0.1, split=True,scores=True,lemmatize=True,pos_filter=('NN','NNS','NNP','NNPS','JJ','JJR','JJS'))

[('policies', 0.19240481053501435),
 ('debt', 0.18738899830538402),
 ('globally', 0.17887859507159526),
 ('countries', 0.17663461614629378),
 ('growth', 0.16748904066315767),
 ('sustainable', 0.1666222230827831),
 ('financial', 0.1582897407996899),
 ('bank', 0.15494729390859266),
 ('financing', 0.1533911340041566),
 ('enhanced', 0.14331676709945917),
 ('management', 0.13999689735802348),
 ('remain', 0.12607457806689876),
 ('including', 0.121085346786063),
 ('productivity', 0.12013352557994032),
 ('risks', 0.11936168118288935),
 ('supports', 0.11198854485646975),
 ('working', 0.111499973924566),
 ('international', 0.11093249720991276),
 ('sector', 0.10217593542622369),
 ('develop', 0.10082542313056617),
 ('strong', 0.09825522455276371),
 ('term', 0.09469573564000999),
 ('quota', 0.08901405308111261),
 ('share', 0.0887439962454196),
 ('resourced', 0.08349955812060139),
 ('address', 0.08071405007659908),
 ('based', 0.08057787489960086),
 ('inclusive', 0.07954313709845132),
 ('lic', 0.0794

### Cleaning Text Data

The raw input text is cleaned off non-printable characters (if any) and turned into lower case.
The processed input text is then tokenized using NLTK library functions. 

In [2]:

import nltk
from nltk import word_tokenize
import string

#nltk.download('punkt')

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    #text = filter(lambda x: x in printable, text) #filter funny characters, if any.
    text = [x for x in text if x in printable]
    text = ''.join(text)
    return text

Cleaned_text = clean(Text)

text = word_tokenize(Cleaned_text)

print ("Tokenized Text: \n")
print (text[:30])

Tokenized Text: 

['the', 'committee', 'expresses', 'its', 'deep', 'gratitude', 'to', 'its', 'former', 'chair', ',', 'mr.', 'agustn', 'carstens', ',', 'for', 'his', 'invaluable', 'contribution', 'to', 'the', 'work', 'of', 'the', 'committee', 'during', '2015-17', ',', 'and', 'extends']


### POS Tagging For Lemmatization

NLTK is again used for <b>POS tagging</b> the input text so that the words can be lemmatized based on their POS tags.

Description of POS tags: 


http://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [3]:
#nltk.download('averaged_perceptron_tagger')
  
POS_tag = nltk.pos_tag(text)
print ("Tokenized Text with POS tags: \n")
print (POS_tag[:20])

Tokenized Text with POS tags: 

[('the', 'DT'), ('committee', 'NN'), ('expresses', 'VBZ'), ('its', 'PRP$'), ('deep', 'JJ'), ('gratitude', 'NN'), ('to', 'TO'), ('its', 'PRP$'), ('former', 'JJ'), ('chair', 'NN'), (',', ','), ('mr.', 'FW'), ('agustn', 'FW'), ('carstens', 'NNS'), (',', ','), ('for', 'IN'), ('his', 'PRP$'), ('invaluable', 'JJ'), ('contribution', 'NN'), ('to', 'TO')]


### Lemmatization

The tokenized text (mainly the nouns and adjectives) is normalized by <b>lemmatization</b>.
In lemmatization different grammatical counterparts of a word will be replaced by single
basic lemma. For example, 'glasses' may be replaced by 'glass'. 

Details about lemmatization: 
    
https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html

In [4]:
#nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

adjective_tags = ['JJ','JJR','JJS']

lemmatized_text = []

for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
        
print ("Text tokens after lemmatization of adjectives and nouns: \n")
print (lemmatized_text[:20])

Text tokens after lemmatization of adjectives and nouns: 

['the', 'committee', 'express', 'it', 'deep', 'gratitude', 'to', 'it', 'former', 'chair', ',', 'mr.', 'agustn', 'carstens', ',', 'for', 'his', 'invaluable', 'contribution', 'to']


### POS tagging for Filtering

The <b>lemmatized text</b> is <b>POS tagged</b> here. The tags will be used for filtering later on.

In [5]:
POS_tag = nltk.pos_tag(lemmatized_text)

print ("Lemmatized text with POS tags: \n")
print (POS_tag[:20])

Lemmatized text with POS tags: 

[('the', 'DT'), ('committee', 'NN'), ('express', 'NN'), ('it', 'PRP'), ('deep', 'JJ'), ('gratitude', 'NN'), ('to', 'TO'), ('it', 'PRP'), ('former', 'JJ'), ('chair', 'NN'), (',', ','), ('mr.', 'FW'), ('agustn', 'FW'), ('carstens', 'NNS'), (',', ','), ('for', 'IN'), ('his', 'PRP$'), ('invaluable', 'JJ'), ('contribution', 'NN'), ('to', 'TO')]


## POS Based Filtering

Any word from the lemmatized text, which isn't a noun, adjective, or gerund (or a 'foreign word'), is here
considered as a <b>stopword</b> (non-content). This is based on the assumption that usually keywords are noun,
adjectives or gerunds. 

Punctuations are added to the stopword list too.

In [6]:
stopwords = []

wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS']#,'VBG','FW'] 

for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords + punctuations

### Complete stopword generation

Even if we remove the aforementioned stopwords, still some extremely common nouns, adjectives or gerunds may
remain which are very bad candidates for being keywords (or part of it). 

An external file constituting a long list of stopwords is loaded and all the words are added with the previous
stopwords to create the final list 'stopwords-plus' which is then converted into a set. 

(Source of stopwords data: https://www.ranks.nl/stopwords)

Stopwords-plus constitute the sum total of all stopwords and potential phrase-delimiters. 

(The contents of this set will be later used to partition the lemmatized text into n-gram phrases. But, for now, I will simply remove the stopwords, and work with a 'bag-of-words' approach. I will be developing the graph using unigram texts as vertices)

In [7]:
with open("long_stopwords.txt", "r") as stopword_file:
    #Source = https://www.ranks.nl/stopwords
    lots_of_stopwords = []

    for line in stopword_file.readlines():
        lots_of_stopwords.append(str(line.strip()))
    
stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

#Stopwords_plus contain total set of all stopwords

### Removing Stopwords 

Removing stopwords from lemmatized_text. 
Processeced_text condtains the result.

In [8]:
processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
print (processed_text[:20])

['committee', 'express', 'deep', 'gratitude', 'chair', 'carstens', 'invaluable', 'contribution', 'committee', '2015-17', 'best', 'general', 'manager', 'bank', 'international', 'settlement', 'committee', 'governor', 'lesetja', 'chairman']


## Vocabulary Creation

Vocabulary will only contain unique words from processed_text.

In [9]:
vocabulary = list(set(processed_text))
print (vocabulary[:10])

['security', 'competitive', 'youth', 'market', 'feasible', 'candid', 'central', 'investment', 'stakeholder', 'laundering']


### Building Graph

TextRank is a graph based model, and thus it requires us to build a graph. Each words in the vocabulary will serve as a vertex for graph. The words will be represented in the vertices by their index in vocabulary list.  

The weighted_edge matrix contains the information of edge connections among all vertices.
I am building wieghted undirected edges.

weighted_edge[i][j] contains the weight of the connecting edge between the word vertex represented by vocabulary index i and the word vertex represented by vocabulary j.

If weighted_edge[i][j] is zero, it means no edge connection is present between the words represented by index i and j.

There is a connection between the words (and thus between i and j which represents them) if the words co-occur within a window of a specified 'window_size' in the processed_text.

The value of the weighted_edge[i][j] is increased by (1/(distance between positions of words currently represented by i and j)) for every connection discovered between the same words in different locations of the text. 

The covered_coocurrences list (which is contain the list of pairs of absolute positions in processed_text of the words whose coocurrence at that location is already checked) is managed so that the same two words located in the same positions in processed_text are not repetitively counted while sliding the window one text unit at a time.

The score of all vertices are intialized to one. 

Self-connections are not considered, so weighted_edge[i][i] will be zero.

In [10]:
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32)
window_size = 5
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])


### Calculating weighted summation of connections of a vertex

inout[i] will contain the sum of all the undirected connections\edges associated withe the vertex represented by i.

In [11]:
inout = weighted_edge.sum(axis=0) ## sum up all edges for each node

### Scoring Vertices

The formula used for scoring a vertex represented by i is:

score[i] = (1-d) + d x [ Summation(j) ( (weighted_edge[i][j]/inout[j]) x score[j] ) ] where j belongs to the list of vertieces that has a connection with i. 

d is the damping factor.

The score is iteratively updated until convergence. 

In [12]:
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print ("Converging at iteration "+str(iter)+"....")
        break


Converging at iteration 28....


In [23]:
scores = zip(vocabulary,score)
sorted(scores, reverse=True,key=lambda t: t[1])

[('imf', 5.4709663),
 ('financial', 5.3652406),
 ('policy', 4.8876224),
 ('growth', 4.6417966),
 ('debt', 4.61518),
 ('country', 3.8036475),
 ('global', 3.4966393),
 ('effort', 3.3871896),
 ('sustainable', 3.1990373),
 ('review', 2.5455973),
 ('trade', 2.519084),
 ('member', 2.498216),
 ('risk', 2.4690132),
 ('rate', 2.422165),
 ('international', 2.4209523),
 ('exchange', 2.4022324),
 ('share', 2.4010665),
 ('strong', 2.3505595),
 ('vulnerability', 2.1975222),
 ('challenge', 2.1331594),
 ('economy', 2.053238),
 ('reform', 1.9809927),
 ('bank', 1.9644489),
 ('financing', 1.9233546),
 ('system', 1.8835435),
 ('monetary', 1.8783695),
 ('program', 1.8771497),
 ('framework', 1.8296367),
 ('tax', 1.8191645),
 ('sector', 1.787791),
 ('sustainability', 1.7673078),
 ('contribution', 1.7141176),
 ('quota', 1.694953),
 ('implementation', 1.6374624),
 ('collaboration', 1.6023885),
 ('general', 1.58237),
 ('consequence', 1.5727794),
 ('imfs', 1.5708778),
 ('committee', 1.5648421),
 ('technology', 1

### Phrase Partiotioning

Paritioning lemmatized_text into phrases using the stopwords in it as delimeters.
The phrases are also candidates for keyphrases to be extracted. 

In [24]:
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords_plus:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords_plus:
        phrase+=str(word)
        phrase+=" "

print ("Partitioned Phrases (Candidate Keyphrases): \n")
print (phrases[:20])

Partitioned Phrases (Candidate Keyphrases): 

[['committee', 'express'], ['deep', 'gratitude'], ['chair'], ['carstens'], ['invaluable', 'contribution'], ['committee'], ['2015-17'], ['best'], ['general', 'manager'], ['bank'], ['international', 'settlement'], ['committee'], ['governor', 'lesetja'], ['chairman'], ['global', 'growth', 'ha'], ['broad-based'], ['strong', 'rebound'], ['investment'], ['trade'], ['risk']]


### Create a list of unique phrases.

Repeating phrases\keyphrase candidates has no purpose here, anymore. 

In [26]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print ("Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases[:20])

Unique Phrases (Candidate Keyphrases): 

[['committee', 'express'], ['deep', 'gratitude'], ['chair'], ['carstens'], ['invaluable', 'contribution'], ['committee'], ['2015-17'], ['best'], ['general', 'manager'], ['bank'], ['international', 'settlement'], ['governor', 'lesetja'], ['chairman'], ['global', 'growth', 'ha'], ['broad-based'], ['strong', 'rebound'], ['investment'], ['trade'], ['risk'], ['term']]


### Thinning the list of candidate-keyphrases.

Removing single word keyphrases-candidates that are present multi-word alternatives. 

In [27]:
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            #if len(phrase)>1 then the current phrase is multi-worded.
            #if the word in vocabulary is present in unique_phrases as a single-word-phrase
            # and at the same time present as a word within a multi-worded phrase,
            # then I will remove the single-word-phrase from the list.
            unique_phrases.remove([word])
            
print ("Thinned Unique Phrases (Candidate Keyphrases): \n")
print (unique_phrases[:20])    

Thinned Unique Phrases (Candidate Keyphrases): 

[['committee', 'express'], ['deep', 'gratitude'], ['chair'], ['carstens'], ['invaluable', 'contribution'], ['2015-17'], ['best'], ['general', 'manager'], ['international', 'settlement'], ['governor', 'lesetja'], ['chairman'], ['global', 'growth', 'ha'], ['broad-based'], ['strong', 'rebound'], ['investment'], ['term'], ['skewed'], ['downside'], ['quarter'], ['financial', 'vulnerability']]


### Scoring Keyphrases

Scoring the phrases (candidate keyphrases) and building up a list of keyphrases\keywords
by listing untokenized versions of tokenized phrases\candidate-keyphrases.
Phrases are scored by adding the score of their members (words\text-units that were ranked by the graph algorithm)


In [28]:
phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

i=0
for keyword in keywords:
    print ("Keyword: '"+str(keyword)+"', Score: "+str(phrase_scores[i]))
    i+=1

Keyword: 'committee express', Score: 2.195065438747406
Keyword: 'deep gratitude', Score: 1.5079360008239746
Keyword: 'chair', Score: 0.803815484046936
Keyword: 'carstens', Score: 0.7890989184379578
Keyword: 'invaluable contribution', Score: 2.4676209688186646
Keyword: '2015-17', Score: 0.6863584518432617
Keyword: 'best', Score: 0.6646711230278015
Keyword: 'general manager', Score: 2.2145882844924927
Keyword: 'international settlement', Score: 3.052969813346863
Keyword: 'governor lesetja', Score: 1.7367520928382874
Keyword: 'chairman', Score: 0.6291137337684631
Keyword: 'global growth ha', Score: 8.74663895368576
Keyword: 'broad-based', Score: 0.6052017211914062
Keyword: 'strong rebound', Score: 2.949370861053467
Keyword: 'investment', Score: 1.049487590789795
Keyword: 'term', Score: 0.6308286786079407
Keyword: 'skewed', Score: 0.637552797794342
Keyword: 'downside', Score: 0.6338039040565491
Keyword: 'quarter', Score: 0.6167439222335815
Keyword: 'financial vulnerability', Score: 7.56276

### Ranking Keyphrases

Ranking keyphrases based on their calculated scores. Displaying top keywords_num no. of keyphrases.

In [30]:
sorted_index = np.flip(np.argsort(phrase_scores),0)
keywords_num = 40
print ("Keywords:\n")
for i in range(0,keywords_num):
    print (str(keywords[sorted_index[i]])+", ",)

Keywords:

high global debt threaten global growth prospect, 
global financial safety net, 
financial sector assessment program, 
lic debt sustainability framework, 
financial stability risk, 
financial sector reform, 
director global policy, 
global growth ha, 
financial sector resilience, 
lic debt vulnerability, 
debt sustainability framework, 
global growth, 
sustainable policy, 
sustainable growth, 
financial vulnerability, 
debt management capacity, 
financial system, 
imf contribution, 
safeguard debt sustainability, 
financial technology, 
financial inclusion, 
financial stability, 
debt vulnerability, 
ieo evaluationthe imf, 
financial market, 
monetary policy, 
resilient international monetary system, 
modern international tax system, 
sustainable financing practice, 
upgrade policy tool, 
fiscal policy, 
productivity growth, 
medium-term growth, 
tailored policy solution, 
excessive global imbalance, 
flexible exchange rate, 
policy tool, 
international tax issue, 
potential

# Input:

Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types of systems and systems of mixed types.

# Extracted Keywords:

* minimal supporting set,  
* minimal generating set,  
* minimal set,  
* linear diophantine equation,  
* nonstrict inequations,  
* strict inequations,  
* system,  
* linear constraint,  
* solution,  
* upper bound, 
